In [2]:
# multivariate multi-step 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [20]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

def model_fit(train, config):
    print("Fitting model...")
    #read config
    n_steps_in = 3
    n_steps_out = 2
    n_features = 2
    X, y = split_sequences(train, n_steps_in, n_steps_out)

    # define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=2000, verbose=0)
    return model
    
def model_predict(model, history, config):
    print("Predicting model...")
    # unpack config
    n_steps_in = 3
    n_steps_out = 2
    # the dataset knows the number of features, e.g. 2
    n_features = 2
    x_input = history #history.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    return yhat[0]

def walk_forward_validation(data, n_test, cfg):
    print("Validation...")
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    print(' > %.3f' % error)
    return [error,predictions]
    
# repeat evaluation of a config
def repeat_evaluate(data, config, n_test, n_repeats=5):
	# fit and evaluate the model n times
    scores = []
    predictions = []
    for _ in range(n_repeats):
        tmp_scores, tmp_predictions = walk_forward_validation(data, n_test, config)
        scores.append(tmp_scores)
        predictions.append(tmp_predictions)
    return [scores,predictions]

# summarize model performance
def summarize_scores(name, scores):
	# print a summary
	scores_m, score_std = mean(scores), std(scores)
	print('%s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std))
	# box and whisker plot
	pyplot.boxplot(scores)
	pyplot.show()

## Prepare Data

In [9]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
data = hstack((in_seq1, in_seq2, out_seq))
data

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [10]:
train, test = train_test_split(data, 2)

In [11]:
test

array([[ 80,  85, 165],
       [ 90,  95, 185]])

Fitting model...


In [24]:
model = model_fit(train,[])
history = [x for x in train]
cfg = []

Fitting model...


In [ ]:
yhat = model_predict(model, history, cfg)

In [35]:
array([1,2,3,4,5,6]).reshape(2,2)

ValueError: cannot reshape array of size 6 into shape (2,2)

In [28]:
n_steps_in = 3
n_features = 2
array(history).reshape((1, n_steps_in, n_features))

ValueError: cannot reshape array of size 21 into shape (1,3,2)

In [26]:
history.reshape((1, n_steps_in, n_features))

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions

In [21]:
# Test
n_test = 3
config = []
scores,predictions = repeat_evaluate(data, config, n_test)
summarize_scores('mlp', scores)

Validation...
Fitting model...
Predicting model...


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6 arrays: [array([[10],
       [15],
       [25]]), array([[20],
       [25],
       [45]]), array([[30],
       [35],
       [65]]), array([[40],
       [45],
       [85]]), array([[ 50],
       [ 55],
       ...

In [14]:
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, 3, 2))
x_input

array([[[70, 75],
        [80, 85],
        [90, 95]]])

In [16]:
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input

array([[70, 75],
       [80, 85],
       [90, 95]])

In [45]:
# multivariate multi-step 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 3
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[186.5056  205.52946 234.89685]]


In [44]:
out_seq

array([[ 25],
       [ 45],
       [ 65],
       [ 85],
       [105],
       [125],
       [145],
       [165],
       [185]])

In [43]:
yhat[0][0]

185.59055

In [40]:
measure_rmse(yhat[0][0],s)

array([[185.59055, 207.3254 ]], dtype=float32)

In [37]:
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
x_input

array([[[70, 75],
        [80, 85],
        [90, 95]]])

In [16]:
X.shape[2]

2

In [22]:
train, test = train_test_split(dataset,5)

In [23]:
X, y = split_sequences(train, n_steps_in, n_steps_out)
X

array([[[10, 15],
        [20, 25],
        [30, 35]]])

In [18]:
y

array([[ 65,  85],
       [ 85, 105],
       [105, 125],
       [125, 145],
       [145, 165],
       [165, 185]])